In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [3]:
from pyspark.sql.dataframe import DataFrame
DataFrame.old_show = getattr(DataFrame, 'old_show', DataFrame.show)
def show(self, n=20, truncate=20, vertical=False):
    self.persist()
    print(self.count())
    self.old_show(n, truncate=truncate, vertical=vertical)
    return self

DataFrame.show = show

import random
def sql(self, q):
    name = 'table_{}'.format(random.randint(0000, 9999))
    self.createOrReplaceTempView(name)
    q = q.format(T=name)
    print(q)
    return df.sql_ctx.sql(q)
DataFrame.sql = sql


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType, StringType, BooleanType
from pyspark.sql import functions as F
spark = SparkSession.builder.getOrCreate()


df = spark.read.json('./jsons/data_200411_194800.json')


In [18]:
import glob 
import os
from pathlib import Path

df = None
for file_name in list(glob.glob('jsons/*.json')):
    print(f'processing {file_name}')
    if os.path.getsize(file_name) == 0:
        print('File is empty, skipping')
        continue
    _df = spark.read.json('./jsons/data_200411_194800.json')
    print(f'{_df.count()} records read from json')
    data = df if not data else data.union(df)


processing jsons/data_200411_194800_.json
1173 records read from json
processing jsons/data_200411_194800.json
1173 records read from json


In [20]:
data.show()

2346
+--------------------+----------+---------------+-------+--------------------+----------+---------+-----------+------+
|        crawl_datetm|est_return|           eval|pattern|        pattern_type|recmd_long|recmd_mid|recmd_short|ticker|
+--------------------+----------+---------------+-------+--------------------+----------+---------+-----------+------+
|2020-04-11 19:47:...|        -9|Near Fair Value|Bullish|            Momentum|      sell|     sell|        buy|   AAN|
|2020-04-11 19:47:...|        33|    Undervalued|Bullish|Commodity Channel...|      sell|     sell|        buy|   ABG|
|2020-04-11 19:47:...|        -2|Near Fair Value|Bullish|Commodity Channel...|       buy|     sell|        buy|  AAPL|
|2020-04-11 19:47:...|        71|    Undervalued|Bullish|Price Crosses Mov...|      sell|      buy|        buy|  ABMD|
|2020-04-11 19:47:...|        14|    Undervalued|Bullish|Commodity Channel...|       buy|     sell|        buy|  ABBV|
|2020-04-11 19:47:...|      null|     Overv

DataFrame[crawl_datetm: string, est_return: bigint, eval: string, pattern: string, pattern_type: string, recmd_long: string, recmd_mid: string, recmd_short: string, ticker: string]

In [21]:
for col in df.columns:
    df.groupby(col).count().sort('count', ascending=False).show(5)

1
+--------------------+-----+
|        crawl_datetm|count|
+--------------------+-----+
|2020-04-11 19:47:...| 1173|
+--------------------+-----+

153
+----------+-----+
|est_return|count|
+----------+-----+
|      null|  192|
|        -6|   25|
|         0|   18|
|         7|   18|
|        -7|   18|
+----------+-----+
only showing top 5 rows

3
+---------------+-----+
|           eval|count|
+---------------+-----+
|     Overvalued|  572|
|    Undervalued|  323|
|Near Fair Value|  278|
+---------------+-----+

3
+-------+-----+
|pattern|count|
+-------+-----+
|Bullish|  859|
|Bearish|  228|
|Neutral|   86|
+-------+-----+

29
+--------------------+-----+
|        pattern_type|count|
+--------------------+-----+
|Commodity Channel...|  497|
|Price Crosses Mov...|  231|
|            Momentum|  137|
|              Gap Up|   57|
|Triple Moving Ave...|   29|
+--------------------+-----+
only showing top 5 rows

3
+----------+-----+
|recmd_long|count|
+----------+-----+
|      sell|  931|